# Optimasi Vehicle Routing Problem <br> Alur Pembuangan Sampah Rayon Surabaya Pusat
<b> Kelompok 08 </b> <br>
Nama Anggota :
1. Nida Aulia Amartika			(5026221095)
2. Isaura Qinthara Heriswan		(5026221146)
3. Devy Relliani Saffiyah		(5026221189)


# Exploratory Data Analysis

In [83]:
%pip install scikit-learn
%pip install folium
%pip install pandas


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [84]:
import pandas as pd
import numpy as np
import random
from sklearn.cluster import KMeans
import folium
import math

In [85]:
df = pd.read_csv('new_data.csv')
df

,No,LPS/Depo,Alamat,Latitude,Longitude,Kapasitas_Sampah(m^3)
0,1,Demak (Kali Butuh),Jalan Demak,-7.253606,112.720422,6
1,2,Pringadi,Jalan Pringadi,-7.252544,112.733072,4
2,3,Penghela,Jalan Penghela,-7.248233,112.733186,3
3,4,Sulung Kali,Jalan Sulung Kali,-7.244014,112.742173,3
4,5,Dupak,Jalan Babatan Dupak,-7.244836,112.727439,2
5,6,Simolawang,Jalan Simolawang,-7.237522,112.753542,1
6,7,Pasar Kapasan,Jalan Simolawang Baru I,-7.239633,112.750247,5
7,8,Tambak Rejo,Jalan Kenjeran (depan Makam Rangkah),-7.243069,112.760314,6
8,9,Simpang Dukuh,Jalan Simpang Dukuh,-7.260503,112.742111,4
9,10,Pasar Genteng,Jalan Genteng Besar,-7.258242,112.740377,4


In [86]:
pool = (-7.26053139, 112.692194)   # Pool Tanjungsari
tpa  = (-7.23739100, 112.60943600) # TPA Benowo

In [87]:
m1 = folium.Map(location=pool, zoom_start=12)
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m1)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m1)
for _, row in df.iterrows():
    folium.Marker(location=(row['Latitude'], row['Longitude']), popup=row['LPS/Depo']).add_to(m1)
m1  # Preview map before VRP

In [88]:
n_vehicles = 11
coords = df[['Latitude','Longitude']].values
kmeans = KMeans(n_clusters=n_vehicles, random_state=42).fit(coords)
df['cluster'] = kmeans.labels_

In [89]:
colors = ['blue', 'purple', 'orange', 'yellow', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue']

# Create a new map centered at the pool
m_cluster = folium.Map(location=pool, zoom_start=12)

# Add depot and TPA markers
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m_cluster)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m_cluster)

# Add markers for each point, colored by cluster
for _, row in df.iterrows():
    folium.Marker(location=(row['Latitude'], row['Longitude']),
                  popup=f"{row['LPS/Depo']} (Cluster {row['cluster']})",
                  icon=folium.Icon(color=colors[row['cluster'] % len(colors)])).add_to(m_cluster)

# Display the map
m_cluster

/var/folders/wy/pyy9phgx4kngk0b6dmzdhvdh0000gn/T/ipykernel_39703/1510036970.py:14: UserWarning: color argument of Icon should be one of: {'white', 'darkgreen', 'darkpurple', 'lightred', 'green', 'orange', 'red', 'darkred', 'pink', 'gray', 'lightgreen', 'beige', 'purple', 'lightblue', 'lightgray', 'blue', 'darkblue', 'cadetblue', 'black'}.
  icon=folium.Icon(color=colors[row['cluster'] % len(colors)])).add_to(m_cluster)


In [90]:
def haversine(a, b):
    R = 6371e3
    φ1, φ2 = math.radians(a[0]), math.radians(b[0])
    Δφ = math.radians(b[0] - a[0])
    Δλ = math.radians(b[1] - a[1])
    x = math.sin(Δφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(Δλ/2)**2
    return R * 2 * math.asin(math.sqrt(x))

In [91]:
def route_distance(route):
    dist = 0
    prev = pool
    for pt in route:
        dist += haversine(prev, pt)
        prev = pt
    dist += haversine(prev, tpa) + haversine(tpa, pool)
    return dist

In [92]:
def order_crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    c1 = [None]*size
    c2 = [None]*size
    c1[a:b], c2[a:b] = p1[a:b], p2[a:b]
    def fill(child, parent):
        pos = b
        for gene in parent[b:] + parent[:b]:
            if gene not in child:
                if pos >= size: pos = 0
                child[pos] = gene
                pos += 1
    fill(c1, p2)
    fill(c2, p1)
    return c1, c2

In [93]:
def mutate(ind):
    if len(ind) > 1:
        i, j = random.sample(range(len(ind)), 2)
        ind[i], ind[j] = ind[j], ind[i]

In [94]:
def ga_cluster(points, pop_size=150, gens=800, elite_size=1):
    n = len(points)
    if n < 2:
        return points

    # initialize population
    population = [random.sample(range(n), n) for _ in range(pop_size)]

    for _ in range(gens):
        # compute fitness for roulette selection
        fitness = [1/(route_distance([points[i] for i in ind]) + 1) for ind in population]
        total = sum(fitness)
        probs = [f/total for f in fitness]

        # select elites (best individuals) to carry forward
        sorted_pop = sorted(
            population,
            key=lambda ind: route_distance([points[i] for i in ind])
        )
        new_pop = [sorted_pop[i].copy() for i in range(elite_size)]

        # fill the rest of new_pop by crossover + mutation
        while len(new_pop) < pop_size:
            # select two parents
            p1, p2 = random.choices(population, probs, k=2)
            # crossover
            if random.random() < 0.8:
                c1, c2 = order_crossover(p1, p2)
            else:
                c1, c2 = p1[:], p2[:]
            # mutate
            if random.random() < 0.2:
                mutate(c1)
            if random.random() < 0.2:
                mutate(c2)

            new_pop.append(c1)
            if len(new_pop) < pop_size:
                new_pop.append(c2)

        population = new_pop

    # return the best route as actual coordinates
    best = min(
        population,
        key=lambda ind: route_distance([points[i] for i in ind])
    )
    return [points[i] for i in best]

In [95]:
m2 = folium.Map(location=pool, zoom_start=12)
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m2)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m2)

# Colors already defined above - no need to redefine

coord_to_name = {
    (row['Latitude'], row['Longitude']): row['LPS/Depo']
    for _, row in df.iterrows()
}
coord_to_name[pool] = "Pool Tanjungsari"
coord_to_name[tpa]  = "TPA Benowo"

for vid in range(n_vehicles):
    # get the raw coordinates in this cluster
    cluster_pts = coords[kmeans.labels_ == vid]
    if len(cluster_pts) == 0:
        continue

    # GA wants a Python list of tuple‐points
    pts_list = [tuple(pt) for pt in cluster_pts]

    # find best ordering of stops
    best_route = ga_cluster(pts_list)

    # build the full trip for plotting
    full_route = [pool] + best_route + [tpa, pool]

    # 1) draw it on the map
    folium.PolyLine(
        locations=full_route,
        color=colors[vid % len(colors)],
        weight=2.5, opacity=0.8,
        popup=f'Vehicle {vid+1}'
    ).add_to(m2)

    # 2) compute the true total distance via route_distance(best_route)
    #    (it already adds pool→stops→TPA→pool)
    total_dist_km = route_distance(best_route) / 1000

    # 3) map coords back to names
    path_names = [coord_to_name[c] for c in full_route]
    print(f"Vehicle {vid+1}: " + " → ".join(path_names))
    print(f"  Total distance: {total_dist_km:.2f} km\n")

# finally show the map
m2

Vehicle 1: Pool Tanjungsari → Pasar Kembang → Pandegiling → Kedondong → Keputran Selatan → Dinoyo → Rumah Sakit Darmo / Ketampon → TPA Benowo → Pool Tanjungsari
  Total distance: 32.77 km

Vehicle 2: Pool Tanjungsari → Giyu → Gayungsari → TPA Benowo → Pool Tanjungsari
  Total distance: 41.98 km

Vehicle 3: Pool Tanjungsari → Alas Malang → Kendung → Babat Jerawat → TPA Benowo → Pool Tanjungsari
  Total distance: 19.83 km

Vehicle 4: Pool Tanjungsari → Kayun → Srikana → Gubeng → Pacar Keling → Legundi Anggrek → TPA Benowo → Pool Tanjungsari
  Total distance: 36.18 km

Vehicle 5: Pool Tanjungsari → Jagir → 3R Jambangan → Gayung Pring → TPA Benowo → Pool Tanjungsari
  Total distance: 40.35 km

Vehicle 6: Pool Tanjungsari → Gebang Putih → Keputih → Medokan Ayu → Semolowaru Bahari → TPA Benowo → Pool Tanjungsari
  Total distance: 50.25 km

Vehicle 7: Pool Tanjungsari → Sulung Kali → Pecindilan → Pasar Kapasan → Tambak Rejo → Simolawang → Wonokusumo Kidul → Nyamplungan → Kertopaten → TPA Beno

In [96]:
colors = ['blue', 'purple', 'orange', 'yellow', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue']

coord_to_name = {
    (row['Latitude'], row['Longitude']): row['LPS/Depo']
    for _, row in df.iterrows()
}
coord_to_name[pool] = "Pool Tanjungsari"
coord_to_name[tpa]  = "TPA Benowo"

for vid in range(n_vehicles):
    # get the raw coordinates in this cluster
    cluster_pts = coords[kmeans.labels_ == vid]
    if len(cluster_pts) == 0:
        continue

    # GA wants a Python list of tuple‐points
    pts_list = [tuple(pt) for pt in cluster_pts]

    # find best ordering of stops
    best_route = ga_cluster(pts_list)

    # build the full trip for plotting
    full_route = [pool] + best_route + [tpa, pool]

    # 1) draw it on the map
    folium.PolyLine(
        locations=full_route,
        color=colors[vid % len(colors)],
        weight=2.5, opacity=0.8,
        popup=f'Vehicle {vid+1}'
    ).add_to(m2)

    # 2) compute the true total distance via route_distance(best_route)
    #    (it already adds pool→stops→TPA→pool)
    total_dist_km = route_distance(best_route) / 1000

    # 3) map coords back to names
    path_names = [coord_to_name[c] for c in full_route]
    print(f"Vehicle {vid+1}: " + " → ".join(path_names))
    print(f"  Total distance: {total_dist_km:.2f} km\n")

# finally show the map
m2

Vehicle 1: Pool Tanjungsari → Pasar Kembang → Pandegiling → Kedondong → Keputran Selatan → Dinoyo → Rumah Sakit Darmo / Ketampon → TPA Benowo → Pool Tanjungsari
  Total distance: 32.77 km

Vehicle 2: Pool Tanjungsari → Giyu → Gayungsari → TPA Benowo → Pool Tanjungsari
  Total distance: 41.98 km

Vehicle 3: Pool Tanjungsari → Alas Malang → Kendung → Babat Jerawat → TPA Benowo → Pool Tanjungsari
  Total distance: 19.83 km

Vehicle 4: Pool Tanjungsari → Kayun → Srikana → Gubeng → Pacar Keling → Legundi Anggrek → TPA Benowo → Pool Tanjungsari
  Total distance: 36.18 km

Vehicle 5: Pool Tanjungsari → Jagir → 3R Jambangan → Gayung Pring → TPA Benowo → Pool Tanjungsari
  Total distance: 40.35 km

Vehicle 6: Pool Tanjungsari → Gebang Putih → Keputih → Medokan Ayu → Semolowaru Bahari → TPA Benowo → Pool Tanjungsari
  Total distance: 50.25 km

Vehicle 7: Pool Tanjungsari → Sulung Kali → Pecindilan → Pasar Kapasan → Tambak Rejo → Simolawang → Wonokusumo Kidul → Nyamplungan → Kertopaten → TPA Beno

# VRP WITH ANT COLONY OPTIMIZATION

In [97]:
# Load kapasitas truk
df_truk = pd.read_csv('kapasitas-truk.csv', sep = ';')
df_truk

,No,Jenis Kendaraan,Volume (m^3)
0,1,Truk Kecil,7
1,2,Truk 3.5 Ton,12
2,3,Arm Roll Truck,18


In [98]:
# Fungsi Ant Colony Optimization sederhana
def ant_colony_tsp(points, n_ants=70, n_iterations=300, alpha=1, beta=2, evaporation=0.4):
    n = len(points)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                dist_matrix[i][j] = haversine(points[i], points[j])
            else:
                dist_matrix[i][j] = 1e-6  # avoid divide by zero

    pheromone = np.ones((n, n))
    best_distance = float('inf')
    best_path = []

    for _ in range(n_iterations):
        all_paths = []
        all_lengths = []

        for _ in range(n_ants):
            unvisited = list(range(n))
            start = random.choice(unvisited)
            path = [start]
            unvisited.remove(start)

            while unvisited:
                curr = path[-1]
                probs = []
                for j in unvisited:
                    tau = pheromone[curr][j] ** alpha
                    eta = (1 / dist_matrix[curr][j]) ** beta
                    probs.append(tau * eta)
                probs = np.array(probs)
                probs /= probs.sum()
                next_node = np.random.choice(unvisited, p=probs)
                path.append(next_node)
                unvisited.remove(next_node)

            length = sum(dist_matrix[path[i]][path[i + 1]] for i in range(n - 1))
            length += haversine(pool, points[path[0]])  # pool to start
            length += haversine(points[path[-1]], tpa)  # end to TPA
            length += haversine(tpa, pool)             # TPA to pool

            all_paths.append(path)
            all_lengths.append(length)

        shortest_idx = np.argmin(all_lengths)
        if all_lengths[shortest_idx] < best_distance:
            best_distance = all_lengths[shortest_idx]
            best_path = all_paths[shortest_idx]

        # Update pheromone
        pheromone *= (1 - evaporation)
        for path, length in zip(all_paths, all_lengths):
            for i in range(n - 1):
                pheromone[path[i]][path[i + 1]] += 1 / length

    return [points[i] for i in best_path], best_distance

In [99]:
# Map baru ACO
m3 = folium.Map(location=pool, zoom_start=12)
total_distance_aco = 0
# Add depot and TPA markers
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m3)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m3)

# Map koordinat ke kapasitas
coord_to_capacity = {
    (row['Latitude'], row['Longitude']): row['Kapasitas_Sampah(m^3)']
    for _, row in df.iterrows()
}

for vid in range(n_vehicles):
    cluster_pts = coords[kmeans.labels_ == vid]
    if len(cluster_pts) == 0:
        continue

    pts_list = [tuple(pt) for pt in cluster_pts]
    capacities = [coord_to_capacity[pt] for pt in pts_list]
    total_volume = sum(capacities)

    # Pilih truk terkecil yang muat
    truk_terpilih = None
    for _, row in df_truk.iterrows():
        if total_volume <= row['Volume (m^3)']:
            truk_terpilih = row
            break
    if truk_terpilih is None:
        truk_terpilih = df_truk.iloc[-1]  # Gunakan terbesar jika semuanya tidak cukup

    # ACO untuk rute terbaik
    best_path, best_dist = ant_colony_tsp(pts_list)
    total_distance_aco += best_dist

    # Bangun rute lengkap
    full_route = [pool] + best_path + [tpa, pool]
    color = colors[vid % len(colors)]
    folium.PolyLine(locations=full_route, color=color, weight=2.5,
                    opacity=0.8, popup=f'Vehicle {vid+1}').add_to(m3)

    path_names = [coord_to_name.get(c, 'Unknown') for c in full_route]


    print(f"[ACO] Vehicle {vid+1}: " + " → ".join(path_names))
    print(f"  Total distance: {best_dist/1000:.2f} km")
    print(f"  Total volume sampah: {total_volume:.1f} m^3")
    print(f"  Jenis truk yang digunakan: {truk_terpilih['Jenis Kendaraan']} (kapasitas {truk_terpilih['Volume (m^3)']} m^3)\n")

print(f"TOTAL DISTANCE ALL VEHICLES [ACO]: {total_distance_aco / 1000:.2f} km")
# Tampilkan map
m3

[ACO] Vehicle 1: Pool Tanjungsari → Pasar Kembang → Pandegiling → Kedondong → Keputran Selatan → Dinoyo → Rumah Sakit Darmo / Ketampon → TPA Benowo → Pool Tanjungsari
  Total distance: 32.77 km
  Total volume sampah: 11.0 m^3
  Jenis truk yang digunakan: Truk 3.5 Ton (kapasitas 12 m^3)

[ACO] Vehicle 2: Pool Tanjungsari → Giyu → Gayungsari → TPA Benowo → Pool Tanjungsari
  Total distance: 41.98 km
  Total volume sampah: 4.0 m^3
  Jenis truk yang digunakan: Truk Kecil (kapasitas 7 m^3)

[ACO] Vehicle 3: Pool Tanjungsari → Alas Malang → Kendung → Babat Jerawat → TPA Benowo → Pool Tanjungsari
  Total distance: 19.83 km
  Total volume sampah: 8.0 m^3
  Jenis truk yang digunakan: Truk 3.5 Ton (kapasitas 12 m^3)

[ACO] Vehicle 4: Pool Tanjungsari → Kayun → Srikana → Gubeng → Pacar Keling → Legundi Anggrek → TPA Benowo → Pool Tanjungsari
  Total distance: 36.18 km
  Total volume sampah: 15.0 m^3
  Jenis truk yang digunakan: Arm Roll Truck (kapasitas 18 m^3)

[ACO] Vehicle 5: Pool Tanjungsari 

# VRP WITH TABU SEARCH ALGORITHM

In [100]:
class TabuSearch:
    def __init__(self, points, capacities, max_vehicle_capacity, tabu_tenure=10, max_iterations=100):
        self.points = points
        self.capacities = capacities
        self.max_capacity = max_vehicle_capacity
        self.tabu_tenure = tabu_tenure
        self.max_iterations = max_iterations
        self.tabu_list = []
        self.best_solution = None
        self.best_distance = float('inf')
        
    def calculate_route_distance(self, route):
        """Calculate total distance for a route including pool->stops->TPA->pool"""
        if not route:
            return 0
        
        dist = haversine(pool, self.points[route[0]])  # pool to first stop
        for i in range(len(route) - 1):
            dist += haversine(self.points[route[i]], self.points[route[i+1]])
        dist += haversine(self.points[route[-1]], tpa)  # last stop to TPA
        dist += haversine(tpa, pool)  # TPA back to pool
        return dist
    
    def generate_initial_solution(self):
        """Generate initial single-route solution visiting all points""" 
        return [list(range(len(self.points)))]  # wrap in list as solution with 1 route

    def get_neighbors(self, solution):
        """Generate neighbors with 2-opt moves within the single route only"""
        neighbors = []
        route = solution[0]  # single route
        
        for i in range(len(route) - 1):
            for j in range(i + 2, len(route)):
                new_route = route[:i+1] + route[i+1:j+1][::-1] + route[j+1:]
                neighbors.append([new_route])  # wrap in list as solution with 1 route
        
        return neighbors
        
    def solution_distance(self, solution):
        """Calculate total distance for entire solution"""
        return sum(self.calculate_route_distance(route) for route in solution)
    
    def is_tabu(self, move):
        """Check if move is in tabu list"""
        return move in self.tabu_list
    
    def add_to_tabu(self, move):
        """Add move to tabu list with tenure management"""
        self.tabu_list.append(move)
        if len(self.tabu_list) > self.tabu_tenure:
            self.tabu_list.pop(0)
    
    def solve(self):
        """Main tabu search algorithm"""
        current_solution = self.generate_initial_solution()
        current_distance = self.solution_distance(current_solution)
        
        self.best_solution = [r[:] for r in current_solution]
        self.best_distance = current_distance
        
        for iteration in range(self.max_iterations):
            neighbors = self.get_neighbors(current_solution)
            
            if not neighbors:
                break
            
            best_neighbor = None
            best_neighbor_distance = float('inf')
            best_move = None
            
            for neighbor in neighbors:
                neighbor_distance = self.solution_distance(neighbor)
                move = str(neighbor)  # Simple move representation
                
                if (neighbor_distance < best_neighbor_distance and 
                    (not self.is_tabu(move) or neighbor_distance < self.best_distance)):
                    best_neighbor = neighbor
                    best_neighbor_distance = neighbor_distance
                    best_move = move
            
            if best_neighbor is None:
                break
            
            current_solution = best_neighbor
            current_distance = best_neighbor_distance
            
            if best_move:
                self.add_to_tabu(best_move)
            
            if current_distance < self.best_distance:
                self.best_solution = [r[:] for r in current_solution]
                self.best_distance = current_distance
        
        return self.best_solution, self.best_distance

In [101]:
# Map baru untuk Tabu Search
m4 = folium.Map(location=pool, zoom_start=12)
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m4)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m4)

# Implementasi Tabu Search untuk setiap cluster
print("=== TABU SEARCH RESULTS ===")
total_distance_tabu = 0

for vid in range(n_vehicles):
    cluster_pts = coords[kmeans.labels_ == vid]
    if len(cluster_pts) == 0:
        continue

    pts_list = [tuple(pt) for pt in cluster_pts]
    capacities = [coord_to_capacity[pt] for pt in pts_list]
    total_volume = sum(capacities)

    # Pilih truk terkecil yang muat
    truk_terpilih = None
    for _, row in df_truk.iterrows():
        if total_volume <= row['Volume (m^3)']:
            truk_terpilih = row
            break
    if truk_terpilih is None:
        truk_terpilih = df_truk.iloc[-1]  # Gunakan terbesar jika semuanya tidak cukup

    # Jalankan Tabu Search
    if len(pts_list) > 1:
        tabu_search = TabuSearch(
            points=pts_list,
            capacities=capacities,
            max_vehicle_capacity=truk_terpilih['Volume (m^3)'],
            tabu_tenure=7,
            max_iterations=50
        )
        
        best_routes, best_distance = tabu_search.solve()
        
        # Visualisasi semua rute untuk kendaraan ini
        vehicle_colors = ['darkred', 'orange', 'purple', 'darkgreen', 'pink']
        
        for route_idx, route in enumerate(best_routes):
            if route:
                # Konversi indeks ke koordinat aktual
                route_coords = [pts_list[i] for i in route]
                full_route = [pool] + route_coords + [tpa, pool]
                
                # Warna untuk sub-rute
                color = vehicle_colors[route_idx % len(vehicle_colors)] if len(best_routes) > 1 else colors[vid % len(colors)]
                
                folium.PolyLine(
                    locations=full_route,
                    color=color,
                    weight=3,
                    opacity=0.8,
                    popup=f'Vehicle {vid+1} Route {route_idx+1}'
                ).add_to(m4)
        
        total_distance_tabu += best_distance
        
        # Output hasil
        path_names_all = []
        for route_idx, route in enumerate(best_routes):
            route_coords = [pts_list[i] for i in route]
            route_names = [coord_to_name.get(c, 'Unknown') for c in [pool] + route_coords + [tpa, pool]]
            path_names_all.append(f"Route {route_idx+1}: " + " → ".join(route_names))
        
        print(f"[TABU] Vehicle {vid+1}:")
        for path_name in path_names_all:
            print(f"  {path_name}")
        print(f"  Total distance: {best_distance/1000:.2f} km")
        print(f"  Total volume sampah: {total_volume:.1f} m^3")
        print(f"  Jenis truk yang digunakan: {truk_terpilih['Jenis Kendaraan']} (kapasitas {truk_terpilih['Volume (m^3)']} m^3)")
     
print(f"TOTAL DISTANCE (TABU SEARCH): {total_distance_tabu/1000:.2f} km")

# Tampilkan map
m4

=== TABU SEARCH RESULTS ===
[TABU] Vehicle 1:
  Route 1: Pool Tanjungsari → Kedondong → Keputran Selatan → Dinoyo → Rumah Sakit Darmo / Ketampon → Pandegiling → Pasar Kembang → TPA Benowo → Pool Tanjungsari
  Total distance: 33.06 km
  Total volume sampah: 11.0 m^3
  Jenis truk yang digunakan: Truk 3.5 Ton (kapasitas 12 m^3)
[TABU] Vehicle 2:
  Route 1: Pool Tanjungsari → Gayungsari → Giyu → TPA Benowo → Pool Tanjungsari
  Total distance: 42.20 km
  Total volume sampah: 4.0 m^3
  Jenis truk yang digunakan: Truk Kecil (kapasitas 7 m^3)
[TABU] Vehicle 3:
  Route 1: Pool Tanjungsari → Kendung → Alas Malang → Babat Jerawat → TPA Benowo → Pool Tanjungsari
  Total distance: 22.50 km
  Total volume sampah: 8.0 m^3
  Jenis truk yang digunakan: Truk 3.5 Ton (kapasitas 12 m^3)
[TABU] Vehicle 4:
  Route 1: Pool Tanjungsari → Kayun → Srikana → Gubeng → Pacar Keling → Legundi Anggrek → TPA Benowo → Pool Tanjungsari
  Total distance: 36.18 km
  Total volume sampah: 15.0 m^3
  Jenis truk yang digunak

# VRP WITH SIMULATED ANNEALING

In [102]:
#SA Function
def simulated_annealing(points, volumes, capacity, initial_temp=500, cooling_rate=0.98, stop_temp=10):
    from functools import lru_cache

    n = len(points)
    current = list(range(n))
    random.shuffle(current)

    def total_distance(route):
        dist = 0
        prev = pool
        for idx in route:
            dist += haversine(prev, points[idx])
            prev = points[idx]
        dist += haversine(prev, tpa) + haversine(tpa, pool)
        return dist

    def is_valid(route):
        load = 0
        for i in route:
            load += volumes[i]
            if load > capacity:
                return False
        return True

    # Cari solusi awal yang valid
    tries = 0
    while not is_valid(current):
        random.shuffle(current)
        tries += 1
        if tries > 1000:
            raise Exception("Gagal mendapatkan solusi awal yang valid")

    best = current[:]
    best_dist = total_distance(best)
    temp = initial_temp
    iter_count = 0

    while temp > stop_temp:
        iter_count += 1
        neighbor = current[:]
        i, j = random.sample(range(n), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]

        if not is_valid(neighbor):
            temp *= cooling_rate
            continue

        curr_dist = total_distance(current)
        neighbor_dist = total_distance(neighbor)
        delta = neighbor_dist - curr_dist

        if delta < 0 or math.exp(-delta / temp) > random.random():
            current = neighbor
            if neighbor_dist < best_dist:
                best = neighbor[:]
                best_dist = neighbor_dist

        temp *= cooling_rate

        if iter_count % 100 == 0:
            print(f"  Iterasi {iter_count}, suhu: {temp:.2f}, best: {best_dist/1000:.2f} km")

    return [points[i] for i in best]

In [103]:
# Map baru untuk Simulated Annealing
m5 = folium.Map(location=pool, zoom_start=12)
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m5)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m5)

# Implementasi Simulated Annealing untuk setiap cluster
print("=== SIMULATED ANNEALING RESULTS ===")
total_distance_sa = 0

for vid in range(n_vehicles):
    cluster_pts = coords[kmeans.labels_ == vid]
    if len(cluster_pts) == 0:
        continue

    pts_list = [tuple(pt) for pt in cluster_pts]
    capacities = [coord_to_capacity[pt] for pt in pts_list]
    total_volume = sum(capacities)

    # Pilih truk terkecil yang muat
    truk_terpilih = None
    for _, row in df_truk.iterrows():
        if total_volume <= row['Volume (m^3)']:
            truk_terpilih = row
            break
    if truk_terpilih is None:
        truk_terpilih = df_truk.iloc[-1]  # Gunakan terbesar jika semuanya tidak cukup

    # Jalankan Simulated Annealing
    if len(pts_list) > 1:
        print(f"\n[SA] Processing Vehicle {vid+1} with {len(pts_list)} stops...")
        try:
            best_route = simulated_annealing(
                points=pts_list,
                volumes=capacities,
                capacity=truk_terpilih['Volume (m^3)'],
                initial_temp=1000,
                cooling_rate=0.95,
                stop_temp=1
            )
            
            # Hitung total jarak rute
            def calculate_total_distance(route_points):
                dist = haversine(pool, route_points[0])
                for i in range(len(route_points) - 1):
                    dist += haversine(route_points[i], route_points[i+1])
                dist += haversine(route_points[-1], tpa)
                dist += haversine(tpa, pool)
                return dist
            
            route_distance = calculate_total_distance(best_route)
            total_distance_sa += route_distance
            
            # Bangun rute lengkap untuk visualisasi
            full_route = [pool] + best_route + [tpa, pool]
            color = colors[vid % len(colors)]
            
            folium.PolyLine(
                locations=full_route,
                color=color,
                weight=3,
                opacity=0.8,
                popup=f'Vehicle {vid+1} (SA)'
            ).add_to(m5)
            
            # Output hasil
            path_names = [coord_to_name.get(c, 'Unknown') for c in full_route]
            print(f"[SA] Vehicle {vid+1}: " + " → ".join(path_names))
            print(f"  Total distance: {route_distance/1000:.2f} km")
            print(f"  Total volume sampah: {total_volume:.1f} m^3")
            print(f"  Jenis truk yang digunakan: {truk_terpilih['Jenis Kendaraan']} (kapasitas {truk_terpilih['Volume (m^3)']} m^3)\n")
            
        except Exception as e:
            print(f"  Error with Vehicle {vid+1}: {e}")
            # Fallback ke rute sederhana
            simple_route = [pool] + pts_list + [tpa, pool]
            folium.PolyLine(
                locations=simple_route,
                color=colors[vid % len(colors)],
                weight=2.5,
                opacity=0.8,
                popup=f'Vehicle {vid+1} (Fallback)'
            ).add_to(m5)
            
            route_distance = haversine(pool, pts_list[0])
            for i in range(len(pts_list) - 1):
                route_distance += haversine(pts_list[i], pts_list[i+1])
            route_distance += haversine(pts_list[-1], tpa) + haversine(tpa, pool)
            total_distance_sa += route_distance
            
            path_names = [coord_to_name.get(c, 'Unknown') for c in simple_route]
            print(f"[SA] Vehicle {vid+1} (Fallback): " + " → ".join(path_names))
            print(f"  Total distance: {route_distance/1000:.2f} km")
            print(f"  Total volume sampah: {total_volume:.1f} m^3")
            print(f"  Jenis truk yang digunakan: {truk_terpilih['Jenis Kendaraan']} (kapasitas {truk_terpilih['Volume (m^3)']} m^3)\n")
    
    else:
        # Hanya satu titik, buat rute sederhana
        single_route = [pool] + pts_list + [tpa, pool]
        folium.PolyLine(
            locations=single_route,
            color=colors[vid % len(colors)],
            weight=2.5,
            opacity=0.8,
            popup=f'Vehicle {vid+1}'
        ).add_to(m5)
        
        route_distance = haversine(pool, pts_list[0]) + haversine(pts_list[0], tpa) + haversine(tpa, pool)
        total_distance_sa += route_distance
        
        path_names = [coord_to_name.get(c, 'Unknown') for c in single_route]
        print(f"[SA] Vehicle {vid+1}: " + " → ".join(path_names))
        print(f"  Total distance: {route_distance/1000:.2f} km")
        print(f"  Total volume sampah: {total_volume:.1f} m^3")
        print(f"  Jenis truk yang digunakan: {truk_terpilih['Jenis Kendaraan']} (kapasitas {truk_terpilih['Volume (m^3)']} m^3)\n")

print(f"TOTAL DISTANCE (SIMULATED ANNEALING): {total_distance_sa/1000:.2f} km")

# Tampilkan map
m5

=== SIMULATED ANNEALING RESULTS ===

[SA] Processing Vehicle 1 with 6 stops...
  Iterasi 100, suhu: 5.92, best: 32.91 km
[SA] Vehicle 1: Pool Tanjungsari → Rumah Sakit Darmo / Ketampon → Dinoyo → Keputran Selatan → Kedondong → Pandegiling → Pasar Kembang → TPA Benowo → Pool Tanjungsari
  Total distance: 32.91 km
  Total volume sampah: 11.0 m^3
  Jenis truk yang digunakan: Truk 3.5 Ton (kapasitas 12 m^3)


[SA] Processing Vehicle 2 with 2 stops...
  Iterasi 100, suhu: 5.92, best: 41.98 km
[SA] Vehicle 2: Pool Tanjungsari → Giyu → Gayungsari → TPA Benowo → Pool Tanjungsari
  Total distance: 41.98 km
  Total volume sampah: 4.0 m^3
  Jenis truk yang digunakan: Truk Kecil (kapasitas 7 m^3)


[SA] Processing Vehicle 3 with 3 stops...
  Iterasi 100, suhu: 5.92, best: 19.83 km
[SA] Vehicle 3: Pool Tanjungsari → Alas Malang → Kendung → Babat Jerawat → TPA Benowo → Pool Tanjungsari
  Total distance: 19.83 km
  Total volume sampah: 8.0 m^3
  Jenis truk yang digunakan: Truk 3.5 Ton (kapasitas 12 m

# Final Algorithm Comparison

In [109]:
# Bandingkan hasil semua algoritma
print("=== COMPREHENSIVE ALGORITHM COMPARISON ===")
print("\n1. Genetic Algorithm (GA):")
print("   - Menggunakan crossover dan mutasi untuk evolusi populasi")
print("   - Baik untuk eksplorasi ruang solusi yang luas")
print("   - Tidak mempertimbangkan kapasitas kendaraan secara eksplisit")

print("\n2. Ant Colony Optimization (ACO):")
print("   - Menggunakan pheromone trails untuk mencari jalur optimal")
print(f"   - Total distance: {total_distance_aco/1000:.2f} km")
print("   - Terinspirasi dari perilaku koloni semut")
print("   - Mempertimbangkan kapasitas kendaraan dalam pemilihan truk")

print("\n3. Tabu Search:")
print("   - Menggunakan memori tabu untuk menghindari local optima")
print("   - Melakukan pencarian lokal dengan pembatasan gerakan")
print(f"   - Total distance: {total_distance_tabu/1000:.2f} km")
print("   - Dapat membuat multiple sub-routes per kendaraan")
print("   - Mempertimbangkan kapasitas kendaraan secara ketat")

print("\n4. Simulated Annealing (SA):")
print("   - Menggunakan probabilitas penerimaan solusi yang menurun")
print("   - Dapat menerima solusi yang lebih buruk untuk menghindari local optima")
print(f"   - Total distance: {total_distance_sa/1000:.2f} km")
print("   - Mempertimbangkan kapasitas kendaraan dengan validasi ketat")

print("\n=== SUMMARY ===")
print("Semua algoritma menggunakan:")
print("- K-Means clustering untuk pembagian wilayah (11 kendaraan)")
print("- Data kapasitas sampah dari new_data.csv")
print("- Data kapasitas truk dari kapasitas-truk.csv")
print("- Rute: Pool Tanjungsari → TPS/Depo → TPA Benowo → Pool Tanjungsari")

print("\nPerbandingan Total Jarak:")
print(f"- Ant Colony Optimization: {total_distance_aco/1000:.2f} km")
print(f"- Tabu Search: {total_distance_tabu/1000:.2f} km")
print(f"- Simulated Annealing: {total_distance_sa/1000:.2f} km")

# Tentukan algoritma terbaik
algo_results = {
    # "Genetic Algorithm": float('inf'),  # Belum dihitung dengan constraint kapasitas
    "Ant Colony Optimization": total_distance_aco,
    "Tabu Search": total_distance_tabu,
    "Simulated Annealing": total_distance_sa
}
# Temukan algoritma dengan total distance paling kecil
best_algo = min(algo_results, key=algo_results.get)
best_distance = algo_results[best_algo]
print(f"\nAlgoritma terbaik: {best_algo} dengan total jarak {best_distance/1000:.2f} km")

=== COMPREHENSIVE ALGORITHM COMPARISON ===

1. Genetic Algorithm (GA):
   - Menggunakan crossover dan mutasi untuk evolusi populasi
   - Baik untuk eksplorasi ruang solusi yang luas
   - Tidak mempertimbangkan kapasitas kendaraan secara eksplisit

2. Ant Colony Optimization (ACO):
   - Menggunakan pheromone trails untuk mencari jalur optimal
   - Total distance: 383.88 km
   - Terinspirasi dari perilaku koloni semut
   - Mempertimbangkan kapasitas kendaraan dalam pemilihan truk

3. Tabu Search:
   - Menggunakan memori tabu untuk menghindari local optima
   - Melakukan pencarian lokal dengan pembatasan gerakan
   - Total distance: 321.43 km
   - Dapat membuat multiple sub-routes per kendaraan
   - Mempertimbangkan kapasitas kendaraan secara ketat

4. Simulated Annealing (SA):
   - Menggunakan probabilitas penerimaan solusi yang menurun
   - Dapat menerima solusi yang lebih buruk untuk menghindari local optima
   - Total distance: 392.60 km
   - Mempertimbangkan kapasitas kendaraan denga